In [1]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
from keras import optimizers

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

2023-10-18 10:32:46.461034: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-18 10:32:46.461112: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-18 10:32:46.461932: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-18 10:32:46.553367: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-18 10:32:47.837285: W tensorflow/compiler/

In [2]:
image_size = [244,244,3]

In [3]:
vgg = VGG16(input_shape=image_size, weights='imagenet', include_top=False)

2023-10-18 10:32:59.098549: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [4]:

for layer in vgg.layers:
  layer.trainable = False

In [5]:
!ls

dataset_eyrc  mymodel.h5	 test_dataset_eyrc
mymodel1.h5   task_2b_evaluator  transfer_learning.ipynb


In [6]:
folders = glob('dataset_eyrc/*')
print(len(folders))

5


In [7]:
x = Flatten()(vgg.output)
prediction = Dense(len(folders), activation='softmax')(x)
model = Model(inputs=vgg.input, outputs=prediction)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 244, 244, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 244, 244, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 244, 244, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 122, 122, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 122, 122, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 122, 122, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 61, 61, 128)       0     

In [8]:

from keras import optimizers


adam = optimizers.Adam()
model.compile(loss='binary_crossentropy',
              optimizer=adam,
              metrics=['accuracy'])
     

In [10]:

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=0,
    width_shift_range=0,
    height_shift_range=0,
    shear_range=0,
    zoom_range=0,
    horizontal_flip=False,
    fill_mode='nearest')
test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=0,
    width_shift_range=0,
    height_shift_range=0,
    shear_range=0,
    zoom_range=0,
    horizontal_flip=False,
    fill_mode='nearest')


In [11]:
train_path = 'dataset_eyrc'
test_path = 'test_dataset_eyrc'

In [12]:
train_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (224, 224),
                                                 batch_size = 16,
                                                 class_mode = 'categorical')
test_set = train_datagen.flow_from_directory(test_path,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 400 images belonging to 5 classes.
Found 10 images belonging to 5 classes.


In [13]:

from datetime import datetime
from keras.callbacks import ModelCheckpoint

In [17]:
checkpoint = ModelCheckpoint(filepath='mymodel1.h5', 
                               verbose=2, save_best_only=True)

callbacks = [checkpoint]

start = datetime.now()

model_history=model.fit(
  train_set,
  validation_data=test_set,
  epochs=5,
  steps_per_epoch=25,
  validation_steps=32,
    callbacks=callbacks ,verbose=2)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Epoch 1/5
25/25 - 29s - loss: 5.1392e-07 - accuracy: 1.0000 - 29s/epoch - 1s/step
Epoch 2/5
25/25 - 30s - loss: 4.1217e-07 - accuracy: 1.0000 - 30s/epoch - 1s/step
Epoch 3/5
25/25 - 30s - loss: 3.5946e-07 - accuracy: 1.0000 - 30s/epoch - 1s/step
Epoch 4/5
25/25 - 29s - loss: 3.1199e-07 - accuracy: 1.0000 - 29s/epoch - 1s/step
Epoch 5/5
25/25 - 30s - loss: 2.7762e-07 - accuracy: 1.0000 - 30s/epoch - 1s/step
Training completed in time:  0:02:27.314895


In [16]:

_# Plot training & validation loss values
plt.plot(model_history.history['accuracy'])
plt.plot(model_history.history['val_accuracy'])
plt.title('CNN Model accuracy values')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

NameError: name 'model_history' is not defined

In [ ]:
from tensorflow.keras.models import load_model
import numpy as np

In [ ]:
loaded_model = load_model('mymodel.h5')
predictions = loaded_model.predict(test_set)
print(predictions)

1/1 [==============================] - 1s 784ms/step
[[0.00000000e+00 2.67467828e-38 9.99999940e-01 1.85315968e-33
  0.00000000e+00]
 [0.00000000e+00 1.15216135e-30 0.00000000e+00 0.00000000e+00
  9.99999940e-01]
 [0.00000000e+00 1.66667662e-34 1.23204024e-30 9.99999940e-01
  1.92385626e-29]
 [8.95908204e-17 6.64233353e-22 6.46958561e-13 9.99999940e-01
  4.93747646e-24]
 [1.10337098e-29 0.00000000e+00 0.00000000e+00 9.99999940e-01
  0.00000000e+00]
 [0.00000000e+00 9.99999940e-01 1.35082715e-36 1.81331884e-20
  7.66048865e-22]
 [1.23631783e-33 9.99999940e-01 9.11032587e-36 4.21686650e-34
  3.29554887e-24]
 [6.95762798e-22 2.17920206e-27 3.52969605e-29 9.99999940e-01
  1.14758305e-28]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  1.00000000e+00]]


In [ ]:
predicted_classes = np.argmax(predictions, axis=1)
class_mapping = {0: 'Combat', 1: 'DestroyedBuildings', 2: 'Fire', 3: 'Humanitarian Aid and rehabilitation', 4: 'Military vehicles and weapons'}

predicted_labels = [class_mapping[prediction] for prediction in predicted_classes]
print(predicted_labels)


['Fire', 'Military vehicles and weapons', 'Humanitarian Aid and rehabilitation', 'Humanitarian Aid and rehabilitation', 'Humanitarian Aid and rehabilitation', 'DestroyedBuildings', 'DestroyedBuildings', 'Humanitarian Aid and rehabilitation', 'Fire', 'Military vehicles and weapons']


In [ ]:
class_names = ['Combat', 'DestroyedBuildings', 'Fire','Humanitarian Aid and rehabilitation','Military vehicles and weapons']

predicted_class_names = [class_names[prediction] for prediction in predicted_classes]
print(predicted_class_names)

['Fire', 'Military vehicles and weapons', 'Humanitarian Aid and rehabilitation', 'Humanitarian Aid and rehabilitation', 'Humanitarian Aid and rehabilitation', 'DestroyedBuildings', 'DestroyedBuildings', 'Humanitarian Aid and rehabilitation', 'Fire', 'Military vehicles and weapons']


In [2]:
import numpy as np
import cv2
from tensorflow.keras.models import load_model

# Define the path to the image

image_path = '/home/zero/eyrc/dataset_eyrc/Combat/1.jpeg'

# Load your model
loaded_model = load_model('mymodel.h5')

# Load and preprocess the image
image = cv2.imread(image_path)
# Resize the image to match the expected input shape of your model
image = cv2.resize(image, (244, 244))
# You may need to further preprocess the image according to your model's requirements.

# Make predictions
predictions = loaded_model.predict(np.expand_dims(image, axis=0))

# Find the predicted class
predicted_classes = np.argmax(predictions, axis=1)

# Define a class mapping
class_mapping = {0: 'combat', 1: 'destroyedbuilding', 2: 'Fire', 3: 'humanitarianaid', 4: 'militaryvehicles'}

# Map the predicted class to its corresponding label
event = [class_mapping[prediction] for prediction in predicted_classes]

# Print the result
print(event)


[ WARN:0@23.280] global loadsave.cpp:248 findDecoder imread_('/home/zero/eyrc/dataset_eyrc/Combat/1.jpeg'): can't open/read file: check file path/integrity


error: OpenCV(4.8.0) /home/conda/feedstock_root/build_artifacts/libopencv_1692668191568/work/modules/imgproc/src/resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'resize'
